In [30]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
k = 64
tree_depth = 6
batch_size = 512
device = 'cuda'
train_data_path = r'<>/mitbih_train.csv'  # replace <> with the correct path of the dataset
test_data_path = r'<>/mitbih_test.csv'  # replace <> with the correct path of the dataset

In [32]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [33]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [34]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [35]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [ ]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.303014755249023 | KNN Loss: 5.731903553009033 | CLS Loss: 1.5711109638214111
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 5.449155807495117 | KNN Loss: 4.73738956451416 | CLS Loss: 0.7117664813995361
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 5.2877278327941895 | KNN Loss: 4.609186172485352 | CLS Loss: 0.6785417795181274
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 5.199596405029297 | KNN Loss: 4.553031921386719 | CLS Loss: 0.6465644836425781
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 5.04219388961792 | KNN Loss: 4.514331340789795 | CLS Loss: 0.5278626084327698
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 5.134302616119385 | KNN Loss: 4.567552089691162 | CLS Loss: 0.5667505860328674
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 5.0261921882629395 | KNN Loss: 4.5067338943481445 | CLS Loss: 0.5194582939147949
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 4.983083724975586 | KNN Loss: 4.485644817352295 | CLS L

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 4.453580856323242 | KNN Loss: 4.280031204223633 | CLS Loss: 0.17354948818683624
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 4.456829071044922 | KNN Loss: 4.278853893280029 | CLS Loss: 0.1779753863811493
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 4.422021389007568 | KNN Loss: 4.252720832824707 | CLS Loss: 0.169300377368927
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 4.439184665679932 | KNN Loss: 4.2957000732421875 | CLS Loss: 0.14348460733890533
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 4.44588041305542 | KNN Loss: 4.277695178985596 | CLS Loss: 0.16818521916866302
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 4.444919109344482 | KNN Loss: 4.2993974685668945 | CLS Loss: 0.1455218642950058
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 4.41594123840332 | KNN Loss: 4.2986297607421875 | CLS Loss: 0.11731164157390594
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 4.403146743774414 | KNN Loss: 4.2885956764221

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 4.271496772766113 | KNN Loss: 4.237403869628906 | CLS Loss: 0.03409278392791748
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 4.4215545654296875 | KNN Loss: 4.311953544616699 | CLS Loss: 0.10960079729557037
Epoch: 007, Loss: 4.3496, Train: 0.9781, Valid: 0.9762, Best: 0.9762
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 4.328602313995361 | KNN Loss: 4.25540018081665 | CLS Loss: 0.07320191711187363
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 4.354703426361084 | KNN Loss: 4.2576446533203125 | CLS Loss: 0.09705885499715805
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 4.363182544708252 | KNN Loss: 4.277888774871826 | CLS Loss: 0.08529369533061981
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 4.294840335845947 | KNN Loss: 4.2391510009765625 | CLS Loss: 0.05568942427635193
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 4.3226799964904785 | KNN Loss: 4.238986492156982 | CLS Loss: 0.08369331061840057
Epoch 8 / 200 | iteratio

Epoch 11 / 200 | iteration 50 / 171 | Total Loss: 4.319055080413818 | KNN Loss: 4.250992298126221 | CLS Loss: 0.06806281954050064
Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 4.287293910980225 | KNN Loss: 4.216714859008789 | CLS Loss: 0.0705791637301445
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 4.3196024894714355 | KNN Loss: 4.24448823928833 | CLS Loss: 0.07511436194181442
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 4.336727142333984 | KNN Loss: 4.268614768981934 | CLS Loss: 0.06811220943927765
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 4.2627034187316895 | KNN Loss: 4.196775913238525 | CLS Loss: 0.06592747569084167
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 4.307506561279297 | KNN Loss: 4.2461066246032715 | CLS Loss: 0.061399850994348526
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 4.333591938018799 | KNN Loss: 4.2286481857299805 | CLS Loss: 0.10494387149810791
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 4.277791500091553 | KNN Loss: 4.23

Epoch 14 / 200 | iteration 130 / 171 | Total Loss: 4.287924289703369 | KNN Loss: 4.241753578186035 | CLS Loss: 0.04617079719901085
Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 4.2644243240356445 | KNN Loss: 4.207264423370361 | CLS Loss: 0.057159729301929474
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 4.306877136230469 | KNN Loss: 4.237769603729248 | CLS Loss: 0.06910748034715652
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 4.267545700073242 | KNN Loss: 4.225411415100098 | CLS Loss: 0.04213442653417587
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 4.248343467712402 | KNN Loss: 4.228784561157227 | CLS Loss: 0.019559090957045555
Epoch: 014, Loss: 4.2794, Train: 0.9854, Valid: 0.9820, Best: 0.9820
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 4.325794219970703 | KNN Loss: 4.262497425079346 | CLS Loss: 0.06329665333032608
Epoch 15 / 200 | iteration 10 / 171 | Total Loss: 4.276278972625732 | KNN Loss: 4.239833354949951 | CLS Loss: 0.0364457443356514
Epoch 15 / 200 

...

Epoch 89 / 200 | iteration 90 / 171 | Total Loss: 4.1525468826293945 | KNN Loss: 4.1374945640563965 | CLS Loss: 0.015052217990159988
Epoch 89 / 200 | iteration 100 / 171 | Total Loss: 4.18501091003418 | KNN Loss: 4.164855003356934 | CLS Loss: 0.020155927166342735
Epoch 89 / 200 | iteration 110 / 171 | Total Loss: 4.166687488555908 | KNN Loss: 4.149980545043945 | CLS Loss: 0.01670677773654461
Epoch 89 / 200 | iteration 120 / 171 | Total Loss: 4.208606719970703 | KNN Loss: 4.180209159851074 | CLS Loss: 0.028397560119628906
Epoch 89 / 200 | iteration 130 / 171 | Total Loss: 4.2185540199279785 | KNN Loss: 4.210280418395996 | CLS Loss: 0.008273464627563953
Epoch 89 / 200 | iteration 140 / 171 | Total Loss: 4.214430809020996 | KNN Loss: 4.202151298522949 | CLS Loss: 0.012279417365789413
Epoch 89 / 200 | iteration 150 / 171 | Total Loss: 4.182023048400879 | KNN Loss: 4.167685031890869 | CLS Loss: 0.014338172972202301
Epoch 89 / 200 | iteration 160 / 171 | Total Loss: 4.17145299911499 | KNN Lo

Epoch 92 / 200 | iteration 160 / 171 | Total Loss: 4.180145740509033 | KNN Loss: 4.1617536544799805 | CLS Loss: 0.01839185319840908
Epoch 92 / 200 | iteration 170 / 171 | Total Loss: 4.194956302642822 | KNN Loss: 4.1919379234313965 | CLS Loss: 0.0030185317154973745
Epoch: 092, Loss: 4.1729, Train: 0.9963, Valid: 0.9865, Best: 0.9878
Epoch 93 / 200 | iteration 0 / 171 | Total Loss: 4.200282573699951 | KNN Loss: 4.175217628479004 | CLS Loss: 0.025065094232559204
Epoch 93 / 200 | iteration 10 / 171 | Total Loss: 4.13863468170166 | KNN Loss: 4.134592533111572 | CLS Loss: 0.0040422710590064526
Epoch 93 / 200 | iteration 20 / 171 | Total Loss: 4.167498588562012 | KNN Loss: 4.149827003479004 | CLS Loss: 0.017671680077910423
Epoch 93 / 200 | iteration 30 / 171 | Total Loss: 4.143689155578613 | KNN Loss: 4.1400861740112305 | CLS Loss: 0.0036030588671565056
Epoch 93 / 200 | iteration 40 / 171 | Total Loss: 4.132369041442871 | KNN Loss: 4.126833915710449 | CLS Loss: 0.005535160191357136
Epoch 93 

Epoch 96 / 200 | iteration 50 / 171 | Total Loss: 4.188086986541748 | KNN Loss: 4.171458721160889 | CLS Loss: 0.01662849821150303
Epoch 96 / 200 | iteration 60 / 171 | Total Loss: 4.154277324676514 | KNN Loss: 4.1363911628723145 | CLS Loss: 0.017885973677039146
Epoch 96 / 200 | iteration 70 / 171 | Total Loss: 4.154818534851074 | KNN Loss: 4.14848518371582 | CLS Loss: 0.006333326920866966
Epoch 96 / 200 | iteration 80 / 171 | Total Loss: 4.169034481048584 | KNN Loss: 4.16160249710083 | CLS Loss: 0.007432049140334129
Epoch 96 / 200 | iteration 90 / 171 | Total Loss: 4.147444248199463 | KNN Loss: 4.139397144317627 | CLS Loss: 0.008047022856771946
Epoch 96 / 200 | iteration 100 / 171 | Total Loss: 4.214555740356445 | KNN Loss: 4.159028053283691 | CLS Loss: 0.05552755668759346
Epoch 96 / 200 | iteration 110 / 171 | Total Loss: 4.18003511428833 | KNN Loss: 4.1660075187683105 | CLS Loss: 0.014027445577085018
Epoch 96 / 200 | iteration 120 / 171 | Total Loss: 4.181146144866943 | KNN Loss: 4.1

Epoch 99 / 200 | iteration 120 / 171 | Total Loss: 4.182658672332764 | KNN Loss: 4.172940254211426 | CLS Loss: 0.009718348272144794
Epoch 99 / 200 | iteration 130 / 171 | Total Loss: 4.150428295135498 | KNN Loss: 4.142909049987793 | CLS Loss: 0.007519348058849573
Epoch 99 / 200 | iteration 140 / 171 | Total Loss: 4.1450347900390625 | KNN Loss: 4.140814781188965 | CLS Loss: 0.004220094531774521
Epoch 99 / 200 | iteration 150 / 171 | Total Loss: 4.186178207397461 | KNN Loss: 4.180509090423584 | CLS Loss: 0.00566928181797266
Epoch 99 / 200 | iteration 160 / 171 | Total Loss: 4.161584854125977 | KNN Loss: 4.156813621520996 | CLS Loss: 0.0047710672952234745
Epoch 99 / 200 | iteration 170 / 171 | Total Loss: 4.174562931060791 | KNN Loss: 4.171230316162109 | CLS Loss: 0.0033327247947454453
Epoch: 099, Loss: 4.1704, Train: 0.9962, Valid: 0.9871, Best: 0.9878
Epoch 100 / 200 | iteration 0 / 171 | Total Loss: 4.171031475067139 | KNN Loss: 4.1543288230896 | CLS Loss: 0.01670273393392563
Epoch 100

Epoch 103 / 200 | iteration 0 / 171 | Total Loss: 4.172691345214844 | KNN Loss: 4.165294170379639 | CLS Loss: 0.007397236302495003
Epoch 103 / 200 | iteration 10 / 171 | Total Loss: 4.156515121459961 | KNN Loss: 4.151280403137207 | CLS Loss: 0.005234846379607916
Epoch 103 / 200 | iteration 20 / 171 | Total Loss: 4.1643967628479 | KNN Loss: 4.157748222351074 | CLS Loss: 0.006648611277341843
Epoch 103 / 200 | iteration 30 / 171 | Total Loss: 4.1371684074401855 | KNN Loss: 4.1332855224609375 | CLS Loss: 0.0038827117532491684
Epoch 103 / 200 | iteration 40 / 171 | Total Loss: 4.189187049865723 | KNN Loss: 4.182706832885742 | CLS Loss: 0.006480189971625805
Epoch 103 / 200 | iteration 50 / 171 | Total Loss: 4.200575351715088 | KNN Loss: 4.194869518280029 | CLS Loss: 0.005705840419977903
Epoch 103 / 200 | iteration 60 / 171 | Total Loss: 4.1659345626831055 | KNN Loss: 4.159405708312988 | CLS Loss: 0.006528736092150211
Epoch 103 / 200 | iteration 70 / 171 | Total Loss: 4.186144828796387 | KNN 

In [37]:
test(model, test_data_iter, device)

tensor(0.9879, device='cuda:0')

In [39]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

  0%|          | 0/43 [00:00<?, ?it/s]

In [42]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [44]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

Number of inliers: 0.9344022657713216


In [45]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [46]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [47]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [48]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [49]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [50]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [51]:
losses = []
accs = []
sparsity = []

In [52]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
Epoch: 00 | Batch: 000 / 040 | Total loss: 1.575 | Reg loss: 0.007 | Tree loss: 1.575 | Accuracy: 0.066406 | 0.07 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 01 | Batch: 000 / 040 | Total loss: 1.395 | Reg loss: 0.004 | Tree loss: 1.395 | Accuracy: 0.689453 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 02 | Batch: 000 / 040 | Total loss: 1.311 | Reg loss: 0.006 | Tree loss: 1.311 | Accuracy: 0.636719 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch:

Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 28 | Batch: 000 / 040 | Total loss: 0.815 | Reg loss: 0.021 | Tree loss: 0.815 | Accuracy: 0.660156 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 29 | Batch: 000 / 040 | Total loss: 0.802 | Reg loss: 0.021 | Tree loss: 0.802 | Accuracy: 0.669922 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 30 | Batch: 000 / 040 | Total loss: 0.781 | Reg loss: 0.021 | Tree loss: 0.781 | Accuracy: 0.675781 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894


Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 56 | Batch: 000 / 040 | Total loss: 0.692 | Reg loss: 0.020 | Tree loss: 0.692 | Accuracy: 0.679688 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 57 | Batch: 000 / 040 | Total loss: 0.701 | Reg loss: 0.020 | Tree loss: 0.701 | Accuracy: 0.697266 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 58 | Batch: 000 / 040 | Total loss: 0.640 | Reg loss: 0.020 | Tree loss: 0.640 | Accuracy: 0.718750 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894


Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 84 | Batch: 000 / 040 | Total loss: 0.692 | Reg loss: 0.020 | Tree loss: 0.692 | Accuracy: 0.681641 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 85 | Batch: 000 / 040 | Total loss: 0.701 | Reg loss: 0.020 | Tree loss: 0.701 | Accuracy: 0.685547 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 86 | Batch: 000 / 040 | Total loss: 0.623 | Reg loss: 0.020 | Tree loss: 0.623 | Accuracy: 0.724609 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894


Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 112 | Batch: 000 / 040 | Total loss: 0.749 | Reg loss: 0.020 | Tree loss: 0.749 | Accuracy: 0.666016 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 113 | Batch: 000 / 040 | Total loss: 0.666 | Reg loss: 0.020 | Tree loss: 0.666 | Accuracy: 0.691406 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 114 | Batch: 000 / 040 | Total loss: 0.646 | Reg loss: 0.020 | Tree loss: 0.646 | Accuracy: 0.707031 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.98404255319148

...

Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 280 | Batch: 000 / 040 | Total loss: 0.635 | Reg loss: 0.020 | Tree loss: 0.635 | Accuracy: 0.755859 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 281 | Batch: 000 / 040 | Total loss: 0.616 | Reg loss: 0.020 | Tree loss: 0.616 | Accuracy: 0.748047 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 282 | Batch: 000 / 040 | Total loss: 0.619 | Reg loss: 0.020 | Tree loss: 0.619 | Accuracy: 0.746094 | 0.046 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.98404255319148

Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 308 | Batch: 000 / 040 | Total loss: 0.633 | Reg loss: 0.020 | Tree loss: 0.633 | Accuracy: 0.753906 | 0.047 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 309 | Batch: 000 / 040 | Total loss: 0.639 | Reg loss: 0.020 | Tree loss: 0.639 | Accuracy: 0.751953 | 0.047 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 310 | Batch: 000 / 040 | Total loss: 0.637 | Reg loss: 0.020 | Tree loss: 0.637 | Accuracy: 0.750000 | 0.047 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.98404255319148

Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 336 | Batch: 000 / 040 | Total loss: 0.558 | Reg loss: 0.020 | Tree loss: 0.558 | Accuracy: 0.765625 | 0.048 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 337 | Batch: 000 / 040 | Total loss: 0.576 | Reg loss: 0.020 | Tree loss: 0.576 | Accuracy: 0.767578 | 0.048 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 338 | Batch: 000 / 040 | Total loss: 0.640 | Reg loss: 0.020 | Tree loss: 0.640 | Accuracy: 0.753906 | 0.048 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.98404255319148

Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 364 | Batch: 000 / 040 | Total loss: 0.681 | Reg loss: 0.020 | Tree loss: 0.681 | Accuracy: 0.720703 | 0.049 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 365 | Batch: 000 / 040 | Total loss: 0.655 | Reg loss: 0.020 | Tree loss: 0.655 | Accuracy: 0.742188 | 0.049 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 366 | Batch: 000 / 040 | Total loss: 0.617 | Reg loss: 0.020 | Tree loss: 0.617 | Accuracy: 0.738281 | 0.049 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.98404255319148

Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 392 | Batch: 000 / 040 | Total loss: 0.668 | Reg loss: 0.020 | Tree loss: 0.668 | Accuracy: 0.734375 | 0.05 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 393 | Batch: 000 / 040 | Total loss: 0.648 | Reg loss: 0.020 | Tree loss: 0.648 | Accuracy: 0.708984 | 0.05 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 394 | Batch: 000 / 040 | Total loss: 0.651 | Reg loss: 0.020 | Tree loss: 0.651 | Accuracy: 0.718750 | 0.05 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894


In [53]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [55]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 4.928571428571429


# Extract Rules

# Accumulate samples in the leaves

In [56]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 14


In [57]:
method = 'greedy'

In [58]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [59]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
============== Pattern 2 ==============
5002
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
3331
============== Pattern 7 ==============
11581
============== Pattern 8 ==============
138
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
403
============== Pattern 13 ==============
============== Pattern 14 ==============
Average comprehensibility: 25.428571428571427
std comprehensibility: 8.666143364630344
var comprehensibility: 75.10204081632654
minimum comprehensibility: 4
maximum comprehensibility: 32


/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))
